In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir("../")

In [3]:
os.getcwd()

'c:\\Projects\\CODEBASE\\DeepCNN_Classifier'

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [22]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_image_size:list
    params_batch_size:int
   

In [23]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories,save_json

class ConfigurationManager:
    def __init__(self,
            config_filepath= CONFIG_FILE_PATH,
            parms_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.parms = read_yaml(parms_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            params_image_size=self.parms.IMAGE_SIZE,
            params_batch_size=self.parms.BATCH_SIZE
        )
        return eval_config

In [27]:
from deepClassifier.utils import save_json
import tensorflow as tf
import time
# creating component
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):
        datagenerator_kwargs =  dict(
                rescale = 1./255,
                validation_split = 0.30 
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
            )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )   
        
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
            
    def evaluation(self):
        model= self.load_model(path=self.config.path_of_model)
        self._valid_generator()
        self.scores = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.scores[0], "accuracy": self.scores[1]}
        save_json(path= Path("scores.json"), data = scores)


In [28]:
try:
    config = ConfigurationManager()
    validation_config = config.get_evaluation_config()
    evaluation  = Evaluation(validation_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise(e)
    

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1228s 3s/step - loss: 3.8316 - accuracy: 0.8602
